# Состояние Онкопомощи в России (2021 год)
Интенсив Антона Ермолина по работе с файлами

## Анотация
Достаточно частая задача практически в любой фирме — это работа с множеством однотипных файлов (например, периодических отчетов), из которых надо сделать сводную информацию для руководства: отчет или дашборд. Если такого рода задача носит постоянный характер, то есть смысл этот процесс автоматизировать: сделать универсальный обработчик, который будет обрабатывать файлы без участия человека и сохранять обработанный результат в том или ином виде: обработанная таблица или дашборд.

Первоисточник всех данных - Отчет (ежегодный) главного онколога Министерства Здравоохранения России академика Каприна А. Д.

Работа состоит из четырех частей:
1. Работа с исходными файлами
    - выгрузка файлов на жесткий диск из Яндекс.диска;
    - ознакомление с информацией, содержащейся в фалах;
    - группировка файлов по смысловой нагрузке;
    - формирование сводных файлов

2. Предварительная обработка полученных сводных файлов

3. Сохранение фалов на жесткий диск, выгрузка копий на Яндекс.диск

4. Работа в Tableau 

## Работа с исходными файлами

In [ ]:
#установка
#pip install pyxll
#pip install --upgrade pip
#pip install pyxll
#pip install pyxll-jupyter
#pip install openpyxl

In [2]:
# импорт библиотек
import yadisk
import os
from tqdm import tqdm
import openpyxl
from openpyxl import Workbook
import sys
import datetime
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import requests

In [3]:
#полученные от яндекса 
app_id = 'здесь от Яндекса app_id'
secret_id = 'здесь от Яндекса secret_id'

#token='здесь от Яндекса token'

In [4]:
y = yadisk.YaDisk(app_id, secret_id)
url = y.get_code_url()
print("Go to the following url: %s" % url)
code = input("Enter the confirmation code: ")
try:
 response = y.get_token(code)
except yadisk.exceptions.BadRequestError:
 print("Bad code")
 sys.exit(1)
y.token = response.access_token
if y.check_token():
 print("Sucessfully received token!")
else:
 print("Something went wrong. Not sure how though...")


Go to the following url: https://oauth.yandex.ru/authorize?response_type=code&client_id=590e92659bcf4cc0ac1915c4a65c4e3f&force_confirm=yes
Enter the confirmation code: 2449242
Sucessfully received token!


In [5]:
# проверяем токен
y.check_token()

True

In [6]:
# создаем список файлов для загрузки
list_of_files = []
for el in list(y.listdir('Test_first')):
    if el['path'].endswith('.xlsx'):
        list_of_files.append(el['path'])

In [7]:
# создаем папку загрузки
load_path = 'C:/Users/User/my_env/test'
if not os.path.exists(load_path):
     os.mkdir(load_path)
os.chdir(load_path)

In [8]:
# загружаем файлы с яндекс диска
for file in tqdm(list_of_files):
     y.download(file.split(':')[1], file.split('/')[-1])

100%|████████████████████████████████████████████████████████████████████████████████| 162/162 [01:55<00:00,  1.40it/s]


In [9]:
#список файлов на локальном компьютере, с которыми будем работать
local_list=os.listdir('C:/Users/User/my_env/test')
  

In [10]:
#файл с полями _субъектРФ, _регион, _полигон_координатами,точечные_координаты
#файл с _субъекатмиРФ, _регион. В названии регионов убираем крайние пробелы
region_dv=pd.read_excel('C:/Users/User/my_env/test/Region_subgect.xlsx')
region_dv.columns=[ 'subject', 'region']
region_dv['region']=region_dv['region'].str.strip()

#файл с регионами и координатами, так же убираем крайние пробелы
region_coor=pd.read_csv('regions_coords.csv')
region_coor['region']=region_coor['region'].str.strip()


region_coor_all=region_dv.merge(region_coor, on='region', how='left')

In [11]:
#1.Функция, которая собирает отчет по ЗНО 
#Список фалов, содержащих zno 
zno_list=[]
subs='Злокачественные_новообразования_в_РФ'
for i in (os.listdir('C:/Users/User/my_env/test')):
    #в папке есть файлы с нмоерами 005, 006, 007, 069 - они не нужны для анализа
    if (subs in i and '00' not in i): 
        if (subs in i and '65' not in i):
            zno_list.append(i) 


#сборная таблица
test_zno=[]
test_all=[]

cals_test=['region','abs','rude_100','standart_100','error_100',
          'М_абсолютное число','М_на 100 тыс.грубый','М_на 100 тыс.стандарт','М_на 100 тыс.ошибка',
          'Ж_абсолютное число','Ж_на 100 тыс.грубый','Ж_на 100 тыс.стандарт','Ж_на 100 тыс.ошибка']
cals_test5=['region','abs','rude_100','standart_100','error_100', 'gender']

flag=0 #флаг для первой сборки    

for k in zno_list:
    test12=pd.read_excel(k, header=[1, 2, 3, 4, 5, 6])
    
    #проверка на количество столбцов 5 или 13 
    if len(test12.columns)==5:
        test12=pd.read_excel(k, header=[1, 2, 3, 4, 5]) #в "гендерных заболеваниях" заголовок на 1 меньше
        test12['gender']='гендерное заболевание'
        test12.columns=cals_test5
        test_zno=test12
    else:    
        test12.columns=cals_test
        t_all=test12.loc[:,['region','abs','rude_100','standart_100','error_100']]
        t_all['gender']='всё население'
        t_man=test12.loc[:,['region','М_абсолютное число','М_на 100 тыс.грубый','М_на 100 тыс.стандарт','М_на 100 тыс.ошибка']]
        t_man['gender']='мужчины'
        t_man.columns=cals_test5
        t_woman=test12.loc[:,['region','Ж_абсолютное число','Ж_на 100 тыс.грубый','Ж_на 100 тыс.стандарт','Ж_на 100 тыс.ошибка']]
        t_woman['gender']='женщины'
        t_woman.columns=cals_test5
        test_zno=pd.concat([t_all,t_man, t_woman])
    
    #добавим столбцы с информацией из заголовков
    test_ex=openpyxl.reader.excel.load_workbook(filename=k)
    table_name=test_ex.active['A1'].value
    table_zc=test_ex.active['A2'].value.lower().split()[0]
    
    #проверка на корректность объединения ячеек в исследуемом файле
    if test_ex.active['B3'].value != None:
        table_year=test_ex.active['B3'].value
    else:
        table_year=test_ex.active['A3'].value.split(': ')[1]
   
    if test_ex.active['B4'].value != None:
            table_local=test_ex.active['B4'].value
    else:
            table_local=test_ex.active['A4'].value.split(': ')[1]
                     
    
    
    #добавляем в сборную таблицу дополнительную информацию
    test_zno['ind']=table_zc
    test_zno['year']=table_year
    test_zno['loc']=table_local
    test_zno['table']=table_name
    test_zno['bzz']='ЗНО'
    
    #проверяем - первый раз сборка или нет через flag
    if flag==0:
        test_all=test_zno
        flag=1
    else:
        test_all=pd.concat([test_all, test_zno], ignore_index=True)
        
#убираем лишние пробелы в названии регионов, добавляем к таблице колонки с координатами, 
#выводим название субъектов  в отдельную колонку
test_all['region']=test_all['region'].str.strip()
test_all=region_coor_all.merge(test_all, on='region', how='left')

#приводим итоговую таблицу к таблице в Паспортизации
test_all=test_all[['region', 'subject', 'gender', 'abs', 'rude_100',
                   'standart_100','error_100', 'ind', 
                   'year','loc','table','bzz', 'polygon_coords']]



test_all.to_excel('ONKO_zno.xlsx')
#первый очтет готов!

In [12]:
#2. Функция, которая собирает отчет по онко58-82
# Список файлов, содержащих
onko_rl=[]
subs='Состояние_онко_помощи_в_РФ'
for i in (os.listdir('C:/Users/User/my_env/test')):
    for k in range(58,86):
        if (subs in i and str(k) in i):
            onko_rl.append(i) 



onko_all=[]
flag=0
onko_col=['region', 'zno_end', 'zno_first', 
          'zno_cont', 'zno_perc', 'met_sur', 
          'met_rad', 'met_med', 'met_com', 'met_h_r']

for k in onko_rl:
    test12=pd.read_excel(k, header=[1, 2])
    test12.columns=onko_col 
               
    #добавим столбцы с информацией из заголовков
    test_ex=openpyxl.reader.excel.load_workbook(filename=k)
    table_name='Т'+test_ex.active['A1'].value.split(' Т')[1]
    table_local=test_ex.active['A1'].value.split('\n')[2].replace(table_name,'').strip()
          
    #добавляем в сборную таблицу дополнительную информацию
    test12['ind']='СВЕДЕНИЯ О ЛЕЧЕНИИ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВАНИЙ (ЗНО), ВПЕРВЫЕ ЗАРЕГИСТРИРОВАННЫХ В 2021 Г., ПОДЛЕЖАЩИХ РАДИКАЛЬНОМУ ЛЕЧЕНИЮ'
    test12['year']='2021'
    test12['loc']=table_local
    test12['table']=table_name
    test12['bzz']='СОП'
    
    #проверяем - первый раз сборка или нет через flag
    if flag==0:
        onko_all=test12
        flag=1
    else:
        onko_all=pd.concat([onko_all, test12], ignore_index=True)

        
#убираем лишние пробелы в названии регионов, добавляем к таблице колонки с координатами, 
#выводим название субъектов  в отдельную колонку
onko_all['region']=onko_all['region'].str.strip()
onko_all=region_coor_all.merge(onko_all, on='region', how='left')

#приводим итоговую таблицу к таблице в Паспортизации
onko_all=onko_all[['region', 'subject', 'zno_end', 'zno_first',
                  'zno_cont', 'zno_perc', 'met_sur', 'met_rad', 
                  'met_med', 'met_com', 'met_h_r', 'ind', 'year', 'loc', 
                  'table', 'bzz', 'polygon_coords']]

#выводим итоговую таблицу


onko_all.to_excel('ONKO_stat.xlsx')

#Второй отчет - готов!

In [13]:
#3. Функция, которая собирает отчет по онко24-51, sheet1

# Список файлов с первым типом заголовка + 32
er1=['24', '25', '26',
    '27','28', '29', 
    '30', '31', '32',
    '39','40', '41',
    '42', '45','46',
     '47', '49','50',
     '51', '52','53']

onko_rl=[]
subs='Состояние_онко_помощи_в_РФ'
for i in (os.listdir('C:/Users/User/my_env/test')):
    for k in er1:
        if (subs in i and str(k) in i):
            onko_rl.append(i) 

onko_all=[]
flag=0
onko_col=['region', 'onko_first', 'activ_perc', 
          'abs_year', '1000_year', 
          '5_abs', '5_perc',
          'index_cont', 'leth']

for k in onko_rl:
    test12=pd.read_excel(k, sheet_name=0, header=[1, 2])
    test12.columns=onko_col 
               
    #добавим столбцы с информацией из заголовков
    test_ex=openpyxl.reader.excel.load_workbook(filename=k)
    test_ex.active=0
    
    #в таблице 32 забыли написать на первом листе, что это таблица 32
    if k=='2021_Таблица_032_Состояние_онко_помощи_в_РФ.xlsx':
        table_name='Таблица 32'
        table_local='Ободочная кишка (С18)'
    else:
        table_name='Т'+test_ex.active['A1'].value.split(' Т')[1].strip()
        table_local=test_ex.active['A1'].value.split('.\n')[1].replace(table_name,'').strip()

    #добавляем в сборную таблицу дополнительную информацию
    test12['ind']='СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ, СОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.'
    test12['year']='2021'
    test12['loc']=table_local
    test12['table']=table_name
    test12['bzz']='СОП'
    
    #проверяем - первый раз сборка или нет через flag
    if flag==0:
        onko_all=test12
        flag=1
    else:
        onko_all=pd.concat([onko_all, test12], ignore_index=True)

#файлы со вторым типом информации
er2=['33', '34', '35',
    '36','37', '38', '43',
    '44', '48']

onko_rl=[]
for i in (os.listdir('C:/Users/User/my_env/test')):
    for k in er2:
        if (subs in i and str(k) in i):
            onko_rl.append(i)
            
for k in onko_rl:
    test12=pd.read_excel(k, sheet_name=0, header=[1, 2])
    test12.columns=onko_col 
               
    #добавим столбцы с информацией из заголовков
    test_ex=openpyxl.reader.excel.load_workbook(filename=k)
    test_ex.active=0
    table_local=test_ex.active['A1'].value.split('\n')[1].strip()
    table_name='Т'+test_ex.active['A1'].value.split(' Т')[1].replace(table_local, '').strip()

    #добавляем в сборную таблицу дополнительную информацию
    test12['ind']='СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ, СОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.'
    test12['year']='2021'
    test12['loc']=table_local
    test12['table']=table_name
    test12['bzz']='СОП'
    
    onko_all=pd.concat([onko_all, test12], ignore_index=True)

#убираем лишние пробелы в названии регионов, добавляем к таблице колонки с координатами, 
#выводим название субъектов  в отдельную колонку
onko_all['region']=onko_all['region'].str.strip()
onko_all=region_coor_all.merge(onko_all, on='region', how='left')

#приводим итоговую таблицу к таблице в Паспортизации
onko_all=onko_all[['region', 'subject', 'onko_first', 
                   'activ_perc', 'abs_year', '1000_year', 
                   '5_abs', '5_perc', 'index_cont',
                   'leth', 'ind', 'year', 'loc',
                   'table', 'bzz', 'polygon_coords']]


onko_all.to_excel('ONKO_cont_1.xlsx')     

#третий отчет готов

In [14]:
#4. функция, которая собирает отчет по онко 24-51, sheet2

onko_rl=[]
table_name_df=[]
subs='Состояние_онко_помощи_в_РФ'
for i in (os.listdir('C:/Users/User/my_env/test')):
    for k in range(24, 52):
        if (subs in i and str(k) in i):
            # переменная s позволяет указать номер таблицы(файла), т.к. в самих фалах беспорядок на "Продолжение таблицы"
            onko_rl.append(i)
            s=i.replace('2021_', '')
            s=s.replace('_Состояние_онко_помощи_в_РФ.xlsx', '')
            s=s.replace('_',' ')
            table_name_df.append(s)
            
onko_all=[]
flag=0
dn=0 #флаг для указателя из какого файла взяли информацию для table_name

onko_col=['region', 'reg_all', 'diagnos', 
          'stad_one', 'stad_two', 
          'stad_three', 'stad_four',
          'stad_none', 'leth_first_year']

for k in onko_rl:
    test12=pd.read_excel(k, sheet_name=1, header=[1, 2, 3])
    test12.columns=onko_col 
    
    #добавим столбцы с информацией из заголовков
    test_ex=openpyxl.reader.excel.load_workbook(filename=k)
    test_ex.active=1
    table_name=table_name_df[dn]
    dn=dn+1
    table_local=(test_ex.active['A1'].value.split('\n')[1]).split(')',1)[0].strip()+')'

    #добавляем в сборную таблицу дополнительную информацию
    test12['ind']='ПОКАЗАТЕЛИ ДИАГНОСТИКИ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВАНИЙ, ВЫЯВЛЕННЫХ В 2021 Г.'
    test12['year']='2021'
    test12['loc']=table_local
    test12['table']=table_name
    test12['bzz']='СОП'
    
    #проверяем - первый раз сборка или нет через flag
    if flag==0:
        onko_all=test12
        flag=1
    else:
        onko_all=pd.concat([onko_all, test12], ignore_index=True)

#убираем лишние пробелы в названии регионов, добавляем к таблице колонки с координатами, 
#выводим название субъектов  в отдельную колонку
onko_all['region']=onko_all['region'].str.strip()
onko_all=region_coor_all.merge(onko_all, on='region', how='left')


#приводим итоговую таблицу к таблице в Паспортизации
onko_all=onko_all[['region', 'subject', 'reg_all', 
                   'diagnos','stad_one','stad_two',
                   'stad_three','stad_four','stad_none',
                   'leth_first_year','ind','year',
                   'loc','table','bzz','polygon_coords']]

onko_cont_2=onko_all

onko_all.to_excel('ONKO_cont_2.xlsx')  
#четвертый отчет готов

## Предобработка фалов
### Таблица onko_zno

Полученная сводная таблица достаточно "грязная". Проведем предобработку, для того чтобы в дальнейшем загрузить файл в Tableau для наглядной презентации

In [15]:
#Предобработка
onko_zno=pd.read_excel('onko_zno.xlsx')
onko_zno.head()

,Unnamed: 0,region,subject,gender,abs,rude_100,standart_100,error_100,ind,year,loc,table,bzz,polygon_coords
0,0,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,всё население,7037,457.96,245.09,3.16,заболеваемость,2021,Злокачественные новообразования - всего (С00-96),Таблица 12,ЗНО,"[[[51.43028730004959,37.853398199829165],[51.4..."
1,1,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,мужчины,3290,462.62,275.97,4.99,заболеваемость,2021,Злокачественные новообразования - всего (С00-96),Таблица 12,ЗНО,"[[[51.43028730004959,37.853398199829165],[51.4..."
2,2,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,женщины,3747,453.95,234.58,4.33,заболеваемость,2021,Злокачественные новообразования - всего (С00-96),Таблица 12,ЗНО,"[[[51.43028730004959,37.853398199829165],[51.4..."
3,3,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,всё население,23,"1,50","0,67","0,15",заболеваемость,2021,Губа (С00),Таблица 13,ЗНО,"[[[51.43028730004959,37.853398199829165],[51.4..."
4,4,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,мужчины,12,"1,69","0,97","0,29",заболеваемость,2021,Губа (С00),Таблица 13,ЗНО,"[[[51.43028730004959,37.853398199829165],[51.4..."


In [16]:
onko_zno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18144 entries, 0 to 18143
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      18144 non-null  int64 
 1   region          18144 non-null  object
 2   subject         18144 non-null  object
 3   gender          18144 non-null  object
 4   abs             18144 non-null  int64 
 5   rude_100        18144 non-null  object
 6   standart_100    18144 non-null  object
 7   error_100       18144 non-null  object
 8   ind             18144 non-null  object
 9   year            18144 non-null  int64 
 10  loc             18144 non-null  object
 11  table           18144 non-null  object
 12  bzz             18144 non-null  object
 13  polygon_coords  18144 non-null  object
dtypes: int64(3), object(11)
memory usage: 1.9+ MB


**Что делаем**
1. Столбец Unnamed - удалем
2. Избавляемся от пробелов в конце строковых значений
3. Столбцы с числами rude_100, standart_100, error_100 - приводим к типу float, предварительно заменив в них симовл ',' на символ '.'

In [17]:
onko_zno.drop('Unnamed: 0', axis= 1 , inplace= True )

#убираем все лишние пробелы в начале и конце значений
onko_zno['subject']=onko_zno['subject'].str.strip()
onko_zno['loc']=onko_zno['loc'].str.strip()

#меняем запятую на точку + преобразовываем в тип float
onko_zno[['rude_100',
          'standart_100', 
          'error_100']]=onko_zno[['rude_100',
                                  'standart_100', 
                                  'error_100']].replace(',','.', regex=True)


onko_zno.head()

,region,subject,gender,abs,rude_100,standart_100,error_100,ind,year,loc,table,bzz,polygon_coords
0,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,всё население,7037,457.96,245.09,3.16,заболеваемость,2021,Злокачественные новообразования - всего (С00-96),Таблица 12,ЗНО,"[[[51.43028730004959,37.853398199829165],[51.4..."
1,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,мужчины,3290,462.62,275.97,4.99,заболеваемость,2021,Злокачественные новообразования - всего (С00-96),Таблица 12,ЗНО,"[[[51.43028730004959,37.853398199829165],[51.4..."
2,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,женщины,3747,453.95,234.58,4.33,заболеваемость,2021,Злокачественные новообразования - всего (С00-96),Таблица 12,ЗНО,"[[[51.43028730004959,37.853398199829165],[51.4..."
3,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,всё население,23,1.50,0.67,0.15,заболеваемость,2021,Губа (С00),Таблица 13,ЗНО,"[[[51.43028730004959,37.853398199829165],[51.4..."
4,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,мужчины,12,1.69,0.97,0.29,заболеваемость,2021,Губа (С00),Таблица 13,ЗНО,"[[[51.43028730004959,37.853398199829165],[51.4..."


In [18]:
#сохраняем новую сводную таблицу
onko_zno.to_excel('onko_zno.xlsx')

### Таблица onko_stat

In [19]:
onko_stat=pd.read_excel('ONKO_stat.xlsx')
onko_stat.head()

,Unnamed: 0,region,subject,zno_end,zno_first,zno_cont,zno_perc,met_sur,met_rad,met_med,met_com,met_h_r,ind,year,loc,table,bzz,polygon_coords
0,0,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,3952,57.1,1418,20.5,53.6,12.7,2.9,27.9,2.9,СВЕДЕНИЯ О ЛЕЧЕНИИ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВ...,2021,Злокачественные новообразования - всего (С00-96),Таблица 58,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
1,1,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,20,87.0,1,4.3,60.0,35.0,0.0,5.0,0.0,СВЕДЕНИЯ О ЛЕЧЕНИИ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВ...,2021,Губа (С00),Таблица 61,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
2,2,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,28,36.4,21,27.3,14.3,7.1,0.0,71.4,7.1,СВЕДЕНИЯ О ЛЕЧЕНИИ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВ...,2021,Полость рта (C01-09),Таблица 62,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
3,3,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,31,27.7,33,29.5,0.0,16.1,0.0,35.5,48.4,СВЕДЕНИЯ О ЛЕЧЕНИИ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВ...,2021,Глотка (С10-13),Таблица 63,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
4,4,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,33,38.8,19,22.4,21.2,0.0,0.0,45.5,33.3,СВЕДЕНИЯ О ЛЕЧЕНИИ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВ...,2021,Пищевод (С15),Таблица 64,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."


In [20]:
onko_stat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2176 entries, 0 to 2175
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      2176 non-null   int64  
 1   region          2176 non-null   object 
 2   subject         2176 non-null   object 
 3   zno_end         2176 non-null   int64  
 4   zno_first       2176 non-null   float64
 5   zno_cont        2176 non-null   int64  
 6   zno_perc        2176 non-null   float64
 7   met_sur         2176 non-null   float64
 8   met_rad         2176 non-null   float64
 9   met_med         2176 non-null   float64
 10  met_com         2176 non-null   float64
 11  met_h_r         2176 non-null   float64
 12  ind             2176 non-null   object 
 13  year            2176 non-null   int64  
 14  loc             2176 non-null   object 
 15  table           2176 non-null   object 
 16  bzz             2176 non-null   object 
 17  polygon_coords  2176 non-null   o

**Что делаем**
1. Столбец Unnamed - удалем
2. Избавляемся от пробелов в конце строковых значений

In [21]:
onko_stat.drop('Unnamed: 0', axis= 1 , inplace= True )
onko_zno['subject']=onko_zno['subject'].str.strip()
onko_zno['loc']=onko_zno['loc'].str.strip()
onko_stat.head()

,region,subject,zno_end,zno_first,zno_cont,zno_perc,met_sur,met_rad,met_med,met_com,met_h_r,ind,year,loc,table,bzz,polygon_coords
0,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,3952,57.1,1418,20.5,53.6,12.7,2.9,27.9,2.9,СВЕДЕНИЯ О ЛЕЧЕНИИ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВ...,2021,Злокачественные новообразования - всего (С00-96),Таблица 58,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
1,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,20,87.0,1,4.3,60.0,35.0,0.0,5.0,0.0,СВЕДЕНИЯ О ЛЕЧЕНИИ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВ...,2021,Губа (С00),Таблица 61,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
2,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,28,36.4,21,27.3,14.3,7.1,0.0,71.4,7.1,СВЕДЕНИЯ О ЛЕЧЕНИИ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВ...,2021,Полость рта (C01-09),Таблица 62,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
3,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,31,27.7,33,29.5,0.0,16.1,0.0,35.5,48.4,СВЕДЕНИЯ О ЛЕЧЕНИИ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВ...,2021,Глотка (С10-13),Таблица 63,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
4,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,33,38.8,19,22.4,21.2,0.0,0.0,45.5,33.3,СВЕДЕНИЯ О ЛЕЧЕНИИ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВ...,2021,Пищевод (С15),Таблица 64,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."


In [22]:
#сохраняем новую сводную таблицу
onko_stat.to_excel('ONKO_stat.xlsx')

### Таблица onko_cont_1

In [23]:
onko_cont_1=pd.read_excel('ONKO_cont_1.xlsx')
onko_cont_1.head()

,Unnamed: 0,region,subject,onko_first,activ_perc,abs_year,1000_year,5_abs,5_perc,index_cont,leth,ind,year,loc,table,bzz,polygon_coords
0,0,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,6134,27.4,44114,2854.9,24836,56.3,7.2,5.1,СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕН...,2021,Злокачественные новообразования - всего (С00-96),Таблица 24,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
1,1,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,17,47.1,357,23.1,243,68.1,21.0,0.6,СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕН...,2021,Губа (С00),Таблица 27,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
2,2,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,71,12.7,348,22.5,152,43.7,4.9,9.4,СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕН...,2021,Полость рта (C01-09),Таблица 28,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
3,3,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,106,5.7,321,20.8,122,38.0,3.0,13.9,СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕН...,2021,Глотка (С10-13),Таблица 29,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
4,4,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,75,8.0,121,7.8,29,24.0,1.6,23.9,СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕН...,2021,Пищевод (С15),Таблица 30,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."


In [24]:
onko_cont_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      2352 non-null   int64  
 1   region          2352 non-null   object 
 2   subject         2352 non-null   object 
 3   onko_first      2352 non-null   int64  
 4   activ_perc      2352 non-null   float64
 5   abs_year        2352 non-null   int64  
 6   1000_year       2352 non-null   float64
 7   5_abs           2352 non-null   int64  
 8   5_perc          2352 non-null   float64
 9   index_cont      2352 non-null   float64
 10  leth            2352 non-null   float64
 11  ind             2352 non-null   object 
 12  year            2352 non-null   int64  
 13  loc             2352 non-null   object 
 14  table           2352 non-null   object 
 15  bzz             2352 non-null   object 
 16  polygon_coords  2352 non-null   object 
dtypes: float64(5), int64(5), object(7

**Что делаем**
1. Столбец Unnamed - удалем
2. Избавляемся от пробелов в конце строковых значений

In [25]:
onko_cont_1.drop('Unnamed: 0', axis= 1 , inplace= True )
onko_cont_1['subject']=onko_zno['subject'].str.strip()
onko_cont_1['loc']=onko_zno['loc'].str.strip()

onko_cont_1.head()

,region,subject,onko_first,activ_perc,abs_year,1000_year,5_abs,5_perc,index_cont,leth,ind,year,loc,table,bzz,polygon_coords
0,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,6134,27.4,44114,2854.9,24836,56.3,7.2,5.1,СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕН...,2021,Злокачественные новообразования - всего (С00-96),Таблица 24,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
1,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,17,47.1,357,23.1,243,68.1,21.0,0.6,СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕН...,2021,Злокачественные новообразования - всего (С00-96),Таблица 27,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
2,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,71,12.7,348,22.5,152,43.7,4.9,9.4,СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕН...,2021,Злокачественные новообразования - всего (С00-96),Таблица 28,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
3,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,106,5.7,321,20.8,122,38.0,3.0,13.9,СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕН...,2021,Губа (С00),Таблица 29,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
4,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,75,8.0,121,7.8,29,24.0,1.6,23.9,СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕН...,2021,Губа (С00),Таблица 30,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."


In [26]:
#сохраняем новую сводную таблицу
onko_cont_1.to_excel('ONKO_cont_1.xlsx')

### Таблица onko_cont_2

In [27]:
onko_cont_2=pd.read_excel('ONKO_cont_2.xlsx')
onko_cont_2.head()

,Unnamed: 0,region,subject,reg_all,diagnos,stad_one,stad_two,stad_three,stad_four,stad_none,leth_first_year,ind,year,loc,table,bzz,polygon_coords
0,0,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,6925,94.5,36.2,26.2,15.3,18.6,3.7,18.7,ПОКАЗАТЕЛИ ДИАГНОСТИКИ ЗЛОКАЧЕСТВЕННЫХ НОВООБР...,2021,Злокачественные новообразования - всего (С00-96),Таблица 024,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
1,1,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,23,100.0,47.8,30.4,8.7,8.7,4.3,15.8,ПОКАЗАТЕЛИ ДИАГНОСТИКИ ЗЛОКАЧЕСТВЕННЫХ НОВООБР...,2021,Губа (С00),Таблица 027,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
2,2,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,77,96.1,6.5,23.4,40.3,29.9,0,25.7,ПОКАЗАТЕЛИ ДИАГНОСТИКИ ЗЛОКАЧЕСТВЕННЫХ НОВООБР...,2021,Полость рта (C01-09),Таблица 028,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
3,3,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,112,98.2,2.7,13.4,47.3,35.7,0.9,22.3,ПОКАЗАТЕЛИ ДИАГНОСТИКИ ЗЛОКАЧЕСТВЕННЫХ НОВООБР...,2021,Глотка (С10-13),Таблица 029,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
4,4,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,85,91.8,8.2,29.4,28.2,31.8,2.4,32.4,ПОКАЗАТЕЛИ ДИАГНОСТИКИ ЗЛОКАЧЕСТВЕННЫХ НОВООБР...,2021,Пищевод (С15),Таблица 030,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."


In [28]:
onko_cont_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2180 entries, 0 to 2179
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       2180 non-null   int64  
 1   region           2180 non-null   object 
 2   subject          2180 non-null   object 
 3   reg_all          2180 non-null   int64  
 4   diagnos          2180 non-null   float64
 5   stad_one         2180 non-null   object 
 6   stad_two         2180 non-null   object 
 7   stad_three       2180 non-null   object 
 8   stad_four        2180 non-null   object 
 9   stad_none        2180 non-null   object 
 10  leth_first_year  2180 non-null   float64
 11  ind              2180 non-null   object 
 12  year             2180 non-null   int64  
 13  loc              2180 non-null   object 
 14  table            2180 non-null   object 
 15  bzz              2180 non-null   object 
 16  polygon_coords   2180 non-null   object 
dtypes: float64(2),

**Что делаем**
1. Столбец Unnamed - удалем
2. Избавляемся от пробелов в конце строковых значений
3. Столбцы с числами stad_one, stad_two, stad_three, stad_four, stad_none - приводим к типу float, предварительно заменив в них симовл ',' на символ '.'

In [29]:
onko_cont_2.drop('Unnamed: 0', axis= 1 , inplace= True )

#убираем все лишние пробелы в начале и конце значений
onko_cont_2['subject']=onko_cont_2['subject'].str.strip()
onko_cont_2['loc']=onko_cont_2['loc'].str.strip()

#меняем запятую на точку + преобразовываем в тип float
onko_cont_2[['stad_one',
          'stad_two', 
          'stad_three',
          'stad_four',
          'stad_none'
         ]]=onko_cont_2[['stad_one',
                                  'stad_two', 
                                  'stad_three',
                                  'stad_four',
                                  'stad_none']].replace(',','.', regex=True)


onko_cont_2.head()

,region,subject,reg_all,diagnos,stad_one,stad_two,stad_three,stad_four,stad_none,leth_first_year,ind,year,loc,table,bzz,polygon_coords
0,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,6925,94.5,36.2,26.2,15.3,18.6,3.7,18.7,ПОКАЗАТЕЛИ ДИАГНОСТИКИ ЗЛОКАЧЕСТВЕННЫХ НОВООБР...,2021,Злокачественные новообразования - всего (С00-96),Таблица 024,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
1,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,23,100.0,47.8,30.4,8.7,8.7,4.3,15.8,ПОКАЗАТЕЛИ ДИАГНОСТИКИ ЗЛОКАЧЕСТВЕННЫХ НОВООБР...,2021,Губа (С00),Таблица 027,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
2,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,77,96.1,6.5,23.4,40.3,29.9,0,25.7,ПОКАЗАТЕЛИ ДИАГНОСТИКИ ЗЛОКАЧЕСТВЕННЫХ НОВООБР...,2021,Полость рта (C01-09),Таблица 028,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
3,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,112,98.2,2.7,13.4,47.3,35.7,0.9,22.3,ПОКАЗАТЕЛИ ДИАГНОСТИКИ ЗЛОКАЧЕСТВЕННЫХ НОВООБР...,2021,Глотка (С10-13),Таблица 029,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."
4,Белгородская область,ЦЕНТРАЛЬНЫЙ ФО,85,91.8,8.2,29.4,28.2,31.8,2.4,32.4,ПОКАЗАТЕЛИ ДИАГНОСТИКИ ЗЛОКАЧЕСТВЕННЫХ НОВООБР...,2021,Пищевод (С15),Таблица 030,СОП,"[[[51.43028730004959,37.853398199829165],[51.4..."


In [30]:
#сохраняем новую сводную таблицу
onko_cont_2.to_excel('ONKO_cont_2.xlsx')

### Загружаем сводные файлы на Яндекс.Диск

In [32]:
y.upload("ONKO_zno.xlsx", "/Test_first/ONKO_zno.xlsx") 
y.upload("ONKO_stat.xlsx", "/Test_first/ONKO_stat.xlsx")
y.upload("ONKO_cont_1.xlsx", "/Test_first/ONKO_cont_1.xlsx") 
y.upload("ONKO_cont_2.xlsx", "/Test_first/ONKO_cont_2.xlsx") 

<SyncResourceLinkObject{'href': 'https://cloud-api.yandex.net/v1/disk/resources?path=disk%3A%2FTest_first%2FONKO_cont_2.xlsx', 'method': 'GET', 'templated': False, 'path': 'disk:/Test_first/ONKO_cont_2.xlsx', 'public_key': None, 'public_url': None, 'type': None, 'file': None}>

## Далее формируем презентацию в Tableau